# Neural Style Transfer

Resources:   
https://towardsdatascience.com/implementing-neural-style-transfer-using-pytorch-fd8d43fb7bfa  
https://pytorch.org/tutorials/advanced/neural_style_tutorial.html

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models

## Model

Here we describe vgg19 model.

Some image of vgg19.

Then its implementation in pytorch and how I am slicing it.

In [ ]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.model = models.vgg19(pretrained=True)

    def forward(self):
        pass


## Loss

### Total Loss

$$
\mathcal{L}_{total}(\vec{p}, \vec{a}, \vec{x}) = \alpha\mathcal{L}_{content}(\vec{p}, \vec{x}) + \beta\mathcal{L}_{style}(\vec{a}, \vec{x})
$$
$ \vec{p} $ - content image  
$ \vec{a} $ - style image   
$ \vec{x} $ - generated image   
$ \alpha $ - content coefficient   
$ \beta $ - style cooefficient 

> Generated image $\vec{x}$ can be either initialized as content image or white noise (random values).

In [ ]:
class TotalLoss(nn.Module):
    def __init__(self, content_features: Tensor, style_features: Tensor, alpha: float = 1., beta: float = 1000.):
        super(TotalLoss, self).__init__()

        self.alpha = alpha
        self.beta = beta

        self.content_loss = ContentLoss(content_features)
        self.style_loss = StyleLoss(style_features)

    def forward(self):
        total_loss = self.alpha * self.content_loss + self.beta * self.style_loss
        return total_loss

### Content Loss

$$
\mathcal{L}_{content}(\vec{p}, \vec{x}) = \dfrac{1}{2} \sum_{i,j}(F_{i,j}^{l} - P_{i,j}^{l})^2
$$

Content loss is a mean squared error between: 
  
$ F_{i,j}^{l} $ - Output of conv layer **$l$** for input (generated) image  
$ P_{i,j}^{l} $ - Output of conv layer **$l$** for content image  

> $ i, j $ represent **i-th** position of the filter at position **j** which implementation-wise doesn't change anything as we take whole outputs of conv layers

There's a small difference in notation when it comes to implementation. In paper, $F^{l}$ is defined as $F^{l} \in \mathbb{R}^{N_l x M_l}$, which means it's a matrix with shape ($N_l$ - number of feature maps, $M_l$ - $width * height$ of feature maps). In contrast, here we have $F^{l} \in \mathbb{R}^{N_l x H_l x W_l}$ which is just a reshaped version of the matrix in the paper.

In [ ]:
class ContentLoss(nn.Module):
    def __init__(self, content_features: Tensor):
        super(ContentLoss, self).__init__()
        
        self.mse = nn.MSELoss()
        self.content_features = content_features

    def forward(self, input_features: Tensor) -> Tensor:
        return self.mse(input_features, self.content_features)

### Style Loss